# Spark 101 Exercises


**1. Create a spark data frame that contains your favorite programming languages.**
  - The name of the column should be ```language```
  - View the schema of the dataframe
  - Output the shape of the dataframe
  - Show the first 5 records in the dataframe
  

In [1]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql.functions import *
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
#create pandas data frame
df = pd.DataFrame(
    dict(language=['Python','SQL','JavaScript', 'Scala', 'Julia', 'Kotlin','TypeScript', 'Java', 'C++', 'R', 'VisualBasic', 'Ruby', 'Java']))

#convert to spark data frame
df = spark.createDataFrame(df)


In [3]:
# view the schema of the dataframe
df.printSchema()

root
 |-- language: string (nullable = true)



In [4]:
#output the shape of the dataframe
print(df.count(), "rows", len(df.columns), "columns")

13 rows 1 columns


In [5]:
#show the first five records
df.show(5)

+----------+
|  language|
+----------+
|    Python|
|       SQL|
|JavaScript|
|     Scala|
|     Julia|
+----------+
only showing top 5 rows



**2. Load the ```mpg``` dataset as a spark dataframe.**

In [6]:
from pydataset import data

#create spark dataframe
mpg = spark.createDataFrame(data("mpg"))
mpg.show(3)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 3 rows



**a. Create 1 column of output that contains a message like the one below for each vehicle:**<br>
```The 1999 audi a4 has a 4 cylinder engine.```

In [7]:
#add a new column to existing dataframe, with formatting to include literals and column name

mpg = mpg.select( '*', concat(
    lit('The '), mpg.year, lit(' '), mpg.manufacturer, lit(' '), mpg.model, lit(' has a '), mpg.cyl, lit(' cylinder engine.')
    ).alias('engine_type')
)

# set truncate to false to keep column content from being cut off
mpg.show(5, truncate=False)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|manufacturer|model|displ|year|cyl|trans     |drv|cty|hwy|fl |class  |engine_type                              |
+------------+-----+-----+----+---+----------+---+---+---+---+-------+-----------------------------------------+
|audi        |a4   |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008 audi a4 has a 4 cylinder engine.|
|audi        |a4   |2.8  |1999|6  |auto(l5)  |f  |16 |26 |p  |compact|The 1999 audi a4 has a 6 cylinder engine.|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+--------------------------

**b. Transform the trans column so that it only contains either manual or auto.**

In [8]:
# use when(expr to identify values beginning with 'auto' and rename them.
# then rename all remaining values to manual
mpg.select(when(expr("trans LIKE 'auto%'"), "auto").otherwise("manual").alias("trans")).show()

+------+
| trans|
+------+
|  auto|
|manual|
|manual|
|  auto|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|  auto|
|manual|
|  auto|
|manual|
|  auto|
|  auto|
|  auto|
|  auto|
|  auto|
+------+
only showing top 20 rows



**3. Load the tips dataset as a spark dataframe.**


In [9]:
from pydataset import data

#create pandas dataframe
tips = pd.DataFrame(data('tips'))

#convert to spark data frame
tips = spark.createDataFrame(data('tips'))
tips.show(3)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
+----------+----+------+------+---+------+----+
only showing top 3 rows



**What percentage of observations are smokers?**

In [10]:
#In order to start using spark SQL, we'll first "register" the table with spark
tips.createOrReplaceTempView("tips")

In [11]:
# Pull with SQL query -- AS creates new column
# Showing only the top line
spark.sql(
    '''
    SELECT ((SELECT COUNT(smoker)
    FROM tips
    WHERE smoker = 'Yes') / 
        (SELECT COUNT(smoker)
        FROM tips)) AS pct_smokers
        FROM tips
    '''
    ).show(1)

+-------------------+
|        pct_smokers|
+-------------------+
|0.38114754098360654|
+-------------------+
only showing top 1 row



**Create a column that contains the tip percentage**


In [12]:
#using SQL commands -- AS tip_pct creates the new column
tips = spark.sql(
    '''
    SELECT *, ROUND((tip / total_bill),2) AS tip_pct
    FROM tips
    '''
)

tips.show(5)

+----------+----+------+------+---+------+----+-------+
|total_bill| tip|   sex|smoker|day|  time|size|tip_pct|
+----------+----+------+------+---+------+----+-------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|   0.06|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|   0.16|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|   0.17|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|   0.14|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|   0.15|
+----------+----+------+------+---+------+----+-------+
only showing top 5 rows



**Calculate the average tip percentage for each combination of sex and smoker.**

In [13]:
# this is similar to a crosstab.  can use a pivot
#Calculate the average tip percentage for each combination of sex and smoker.

#Use groupby and pivot table to efficiently calculate these values
tips.groupby("sex").pivot('smoker').mean('tip_pct').show()

+------+-------------------+-------------------+
|   sex|                 No|                Yes|
+------+-------------------+-------------------+
|Female|0.15685185185185185| 0.1824242424242424|
|  Male|0.16103092783505155|0.15283333333333332|
+------+-------------------+-------------------+



**4. Use the seattle weather dataset referenced in the lesson to answer the questions below.**

In [14]:
from vega_datasets import data as vdata

# format all dates to string datatypes
weather= vdata.seattle_weather().assign(date=lambda df: df.date.astype(str))

#create spark dataframe
weather = spark.createDataFrame(weather)
weather.show(3)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 3 rows



**- Convert the temperatures to farenheight.**

In [15]:
# use formula to convert original celsius temps to farenheit and create new columns to hold them
weather = weather.select("*",
    round((weather.temp_max * 1.8 + 32),1).alias('temp_max_F'),
    round((weather.temp_min * 1.8 + 32),1).alias('temp_min_F')
)

weather.show()

+----------+-------------+--------+--------+----+-------+----------+----------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|
+----------+-------------+--------+--------+----+-------+----------+----------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      39.9|      36.0|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      45.0|      37.0|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|      50.0|      37.0|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|      48.9|      41.0|
|2012-01-10|          1.0|     6.1|     

**- Which month has the most rain, on average?**

In [16]:

# import necessary spark functions
from pyspark.sql.functions import month, year, quarter

#create new column for month

weather = weather.withColumn('month', month('date'))
weather.show()

+----------+-------------+--------+--------+----+-------+----------+----------+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|month|
+----------+-------------+--------+--------+----+-------+----------+----------+-----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|    1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|    1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|    1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|    1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|    1|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      39.9|      36.0|    1|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      45.0|      37.0|    1|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|      50.0|      37.0|    1|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   ra

In [17]:
#create new column for year

weather = weather.withColumn('year', year('date'))
weather.show()

+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|month|year|
+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|    1|2012|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|    1|2012|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|    1|2012|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|    1|2012|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|    1|2012|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      39.9|      36.0|    1|2012|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      45.0|      37.0|    1|2012|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|      50.0|      37.0|    1|2012|

In [18]:
#create new column for quarter

weather = weather.withColumn('quarter', quarter('date'))
weather.show()

+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|month|year|quarter|
+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      55.0|      41.0|    1|2012|      1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      51.1|      37.0|    1|2012|      1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      53.1|      45.0|    1|2012|      1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      54.0|      42.1|    1|2012|      1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      48.0|      37.0|    1|2012|      1|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      39.9|      36.0|    1|2012|      1|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      45.0|      37.0|    1|2012|      1|
|2012-01-0

In [19]:
#group by month & aggregate by average precipitation
# sort by descending and show only the top (greatest) value
weather.groupby('month').avg('precipitation').sort(desc('avg(precipitation)')).show(1)


+-----+------------------+
|month|avg(precipitation)|
+-----+------------------+
|   11| 5.354166666666667|
+-----+------------------+
only showing top 1 row



**- Which year was the windiest?**

In [20]:
# group by year  & aggregate by average wind value
# sort by descending and show only the top (greatest) value
weather.groupby('year').avg('wind').sort(desc('avg(wind)')).show(1)

+----+-----------------+
|year|        avg(wind)|
+----+-----------------+
|2012|3.400819672131148|
+----+-----------------+
only showing top 1 row



**- What is the most frequent type of weather in January?**

In [21]:
#create a crosstab by month (January)
weather.where(weather.month == 1).crosstab('month', 'weather').show()

+-------------+-------+---+----+----+---+
|month_weather|drizzle|fog|rain|snow|sun|
+-------------+-------+---+----+----+---+
|            1|     10| 38|  35|   8| 33|
+-------------+-------+---+----+----+---+



**- What is the average high and low temperature on sunny days in July in 2013 and 2014?**

In [22]:
sunny_days = weather\
    .where(weather.weather == "sun")\
    .where(weather.month == 7)\
    .where((weather.year == 2013) | (weather.year == 2014))

sunny_days.show()

+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|temp_max_F|temp_min_F|month|year|quarter|
+----------+-------------+--------+--------+----+-------+----------+----------+-----+----+-------+
|2013-07-01|          0.0|    31.7|    18.3| 2.3|    sun|      89.1|      64.9|    7|2013|      3|
|2013-07-02|          0.0|    28.3|    15.6| 3.0|    sun|      82.9|      60.1|    7|2013|      3|
|2013-07-03|          0.0|    26.1|    16.7| 3.2|    sun|      79.0|      62.1|    7|2013|      3|
|2013-07-05|          0.0|    23.3|    13.9| 2.6|    sun|      73.9|      57.0|    7|2013|      3|
|2013-07-06|          0.0|    26.1|    13.3| 2.2|    sun|      79.0|      55.9|    7|2013|      3|
|2013-07-07|          0.0|    23.9|    13.9| 2.9|    sun|      75.0|      57.0|    7|2013|      3|
|2013-07-08|          0.0|    26.7|    13.3| 2.8|    sun|      80.1|      55.9|    7|2013|      3|
|2013-07-0

In [23]:
sunny_days.select(avg('temp_max_F').alias('avg_max_temp_F'), avg('temp_min_F').alias('avg_min_temp_F')).show()

+-----------------+------------------+
|   avg_max_temp_F|    avg_min_temp_F|
+-----------------+------------------+
|80.29423076923078|57.513461538461556|
+-----------------+------------------+



**- What percentage of days were rainy in q3 of 2015?**

In [24]:
# create variable for months 7,8,9 to represent third quarter
q3 = weather.where(weather.year == 2015).where(weather.quarter == 3) 

# divide to find percentage
q3.where(q3.precipitation > 0).count() / q3.count()


0.18478260869565216

**- For each year, find what percentage of days it rained (had non-zero precipitation).**

In [25]:
#create dataframe of rainy days grouped by year
rainy_days = weather.where(weather.precipitation > 0).groupby('year').count()

In [26]:
rainy_days.select(round(rainy_days['count'] / 365,2).alias('pct_year_with_rain'), 'year').show()

+------------------+----+
|pct_year_with_rain|year|
+------------------+----+
|              0.39|2015|
|              0.42|2013|
|              0.41|2014|
|              0.48|2012|
+------------------+----+

